In [1]:
# Dependencies and Setup
import pandas as pd
from functools import reduce

# Files to Load
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
df_school = pd.read_csv(school_data)
df_student = pd.read_csv(student_data)

# Combine the data into a single dataset
df = pd.merge(df_student, df_school, how="left", on=["school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#District Summary
schools_total = df.school_name.nunique()
students_total = f"{df['Student ID'].count():,}"
budget_total = f"${df.budget.unique().sum():,.2f}"
math_avg = f"{df.math_score.mean():.2f}"
reading_avg = f"{df.reading_score.mean():.2f}"
math_passed = f"{(df.math_score.loc[df.math_score >= 70].count())/(df.math_score.count())*100:.2f}"
reading_passed = f"{(df.reading_score.loc[df.reading_score >= 70].count())/(df.reading_score.count())*100:.2f}"
overall_passed = df.loc[(df.math_score >= 70) & (df.reading_score >= 70)]
overall_passed = f"{overall_passed.math_score.count()/df.math_score.count()*100:.2f}"

district_table = pd.DataFrame([[schools_total, students_total, budget_total, math_avg, reading_avg, math_passed, reading_passed, overall_passed]],
                             columns=['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                      '% Overall Passing'])
district_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#df aggregated at school level
df_school.rename(columns={'school_name': 'School Name', 'type': 'School Type', 'size': 'Total Students', 'budget': 'Total School Budget'}, inplace=True)
df_school['Per Student Budget'] = df_school.apply(lambda row: row['Total School Budget']/row['Total Students'], axis=1)
df_school.sort_values(['School Name'], inplace=True, ignore_index=True)
df_school.reset_index(drop=True, inplace=True)

In [4]:
#Performing calculations on combined DF
dfs = df
dfs.sort_values('school_name', inplace=True)
dfs.rename(columns={'school_name': 'School Name', 'type': 'School Type', 'size': 'Total Students', 'budget': 'Total School Budget'}, inplace=True)
grp_sn = dfs.groupby(['School Name'])
average_scores = grp_sn[['math_score', 'reading_score']].mean().reset_index()
dfs['math_pass'] = dfs.math_score.apply(lambda x: 1 if x >= 70 else 0)
dfs['reading_pass'] = dfs.reading_score.apply(lambda x: 1 if x >= 70 else 0)
dfs['overall_pass'] = dfs.apply(lambda row: 1 if (row.math_score >= 70) & (row.reading_score >= 70) else 0, axis=1)
math_pass = (grp_sn.math_pass.sum()/grp_sn.math_pass.count()*100).reset_index()
reading_pass = (grp_sn.reading_pass.sum()/grp_sn.reading_pass.count()*100).reset_index()
overall_pass = (grp_sn.overall_pass.sum()/grp_sn.overall_pass.count()*100).reset_index()

In [5]:
#School Summary
df_list = [df_school, average_scores, math_pass, reading_pass, overall_pass]
school_summary = reduce(lambda left, right: pd.merge(left, right, on=['School Name'], how='left'), df_list)
school_summary.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score',\
               'math_pass': '% Passing Math', 'reading_pass': '% Passing Reading', 'overall_pass': '% Overall Passing'}, inplace=True)
school_summary.drop(columns=['School ID'], inplace=True)
school_summary.set_index('School Name', inplace=True)

#Formatting
school_summary_formatted = pd.DataFrame(school_summary).reset_index()
school_summary_formatted["Total School Budget"] = school_summary_formatted["Total School Budget"].map("${:,.2f}".format)
school_summary_formatted["Per Student Budget"] = school_summary_formatted["Per Student Budget"].map("${:,.2f}".format)


school_summary_formatted

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
#Top Five Schools (% Overall Passing)
school_summary_formatted.sort_values(['% Overall Passing'], ascending=False, inplace=True)
school_summary_formatted.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
#Bottom Five Schools (% Overall Passing)
school_summary_formatted.sort_values(['% Overall Passing'], inplace=True)
school_summary_formatted.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
#Math Score for each School by grade level (9th, 10th, 11th, 12th)
dfsm = dfs
grp_snm = dfs.groupby(['School Name'])

#math
dfsm['9th'] = dfsm.loc[dfsm.grade == '9th', 'math_score']
dfsm['10th'] = dfsm.loc[dfsm.grade == '10th', 'math_score']
dfsm['11th'] = dfsm.loc[dfsm.grade == '11th', 'math_score']
dfsm['12th'] = dfsm.loc[dfsm.grade == '12th', 'math_score']
nineth = grp_snm['9th'].mean().reset_index()
tenth = grp_snm['10th'].mean().reset_index()
eleventh = grp_snm['11th'].mean().reset_index()
twelveth = grp_snm['12th'].mean().reset_index()

m_merge1 = pd.merge(nineth, tenth, how='left', on=['School Name', 'School Name'])
m_merge2 = pd.merge(m_merge1, eleventh, how='left', on=['School Name', 'School Name'])
math_avg_by_grade = pd.merge(m_merge2, twelveth, how='left', on=['School Name', 'School Name'])
math_avg_by_grade.reset_index(drop=True, inplace=True)
math_avg_by_grade
math_avg_by_grade.set_index
math_avg_by_grade

,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
#Reading Score for each School by grade level (9th, 10th, 11th, 12th)
dfsr = dfs
grp_snr = dfs.groupby(['School Name'])

#reading
dfsr['9th'] = dfsr.loc[dfsr.grade == '9th', 'reading_score']
dfsr['10th'] = dfsr.loc[dfsr.grade == '10th', 'reading_score']
dfsr['11th'] = dfsr.loc[dfsr.grade == '11th', 'reading_score']
dfsr['12th'] = dfsr.loc[dfsr.grade == '12th', 'reading_score']
nineth = grp_snr['9th'].mean().reset_index()
tenth = grp_snr['10th'].mean().reset_index()
eleventh = grp_snr['11th'].mean().reset_index()
twelveth = grp_snr['12th'].mean().reset_index()

dfr_list = [nineth, tenth, eleventh, twelveth]
reading_avg_by_grade = reduce(lambda left, right: pd.merge(left, right, on=['School Name'], how='left'), dfr_list)
reading_avg_by_grade.reset_index(drop=True, inplace=True)
reading_avg_by_grade

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending: Values consistent with homework key but not weighted appropriately

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
#Scores by School Spending (Produces a table identical to example in the homework)
spending = pd.DataFrame(school_summary[['Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']])
spending.reset_index(inplace=True, drop=True)
max = spending['Per Student Budget'].max()
labels = ['<$584', '$585-629', '$630-644', '$645-675']
bins = [0, 584, 629, 644, max]
spending['Spending Ranges (Per Student)'] = pd.cut(spending['Per Student Budget'], bins, labels=labels , include_lowest=True)
spending.set_index('Spending Ranges (Per Student)', inplace=True)
spending = spending.groupby('Spending Ranges (Per Student)').mean()
for column in spending.columns:
    spending[str(column)] =  spending[str(column)].map("{:.2f}".format)
spending.drop(columns='Per Student Budget', inplace=True)

spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Spending: Corrected values


In [11]:
"""The following code produces averages that are more accurate, but are slightly different from the values in the homework answer key.
This is because the homework answer key shows averages per spending category calculated on data aggregated at the school level, and the
data were not weighted by school size (as shown above). The following code calculates averages for each spending category based on student-level data."""

df.rename(columns={'school_name': 'School Name', 'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score', 'type': 'School Type', 'size': 'Total Students', 'budget': 'Total School Budget'}, inplace=True)
df['Per Student Budget'] = df['Total School Budget']/df['Total Students']
df['% Passing Math'] = df['Average Math Score'].apply(lambda x: 100 if x >= 70 else 0)
df['% Passing Reading'] = df['Average Reading Score'].apply(lambda x: 100 if x >= 70 else 0)
df['% Overall Passing'] = df.apply(lambda row: 100 if (row['Average Math Score'] >= 70) & (row['Average Reading Score'] >= 70) else 0, axis=1)
max = df['Per Student Budget'].max()
df['Spending Ranges (Per Student)'] = pd.cut(df['Per Student Budget'], bins, labels=labels , include_lowest=True)
df.set_index('Spending Ranges (Per Student)', inplace=True)
spending_corrected = pd.DataFrame(df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']])
spending_corrected = spending_corrected.groupby(['Spending Ranges (Per Student)']).mean()
for column in spending_corrected.columns:
    spending_corrected[str(column)] =  spending_corrected[str(column)].map("{:.2f}".format)
spending_corrected

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,79.98,82.31,79.11,88.51,70.94
$630-644,77.82,81.30,70.62,82.60,58.84
$645-675,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
#Scores by School Size
size = pd.DataFrame(school_summary[['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']])
size.reset_index(inplace=True, drop=True)
labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
bins = [0, 1000, 2000, 5000]
size['School Size'] = pd.cut(size['Total Students'], bins, labels=labels , include_lowest=True)
size.set_index('School Size', inplace=True)
size = size.groupby('School Size').mean()
for column in size.columns:
    size[str(column)] =  size[str(column)].map("{:.2f}".format)
size.drop(columns='Total Students', inplace=True)
size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [13]:
#Scores by School Type
type = pd.DataFrame(school_summary[['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']])
type.reset_index(inplace=True, drop=True)
type.set_index('School Type', inplace=True)
type = type.groupby('School Type').mean()
for column in type.columns:
    type[str(column)] =  type[str(column)].map("{:.2f}".format)
type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
